<a href="https://colab.research.google.com/github/marriamaslova/compling_nlp_hse_course/blob/master/notebooks/spelling/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

Ответ с демонстрацией результата

In [ ]:
!pip install razdel tqdm

In [ ]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
!pip install textdistance

In [ ]:
import textdistance

In [ ]:
from difflib import get_close_matches

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [ ]:
from operator import itemgetter, attrgetter, methodcaller

In [ ]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [ ]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [ ]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def keyFunc(elem):
    return elem[2]

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [ ]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [ ]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    elems = []
    for elem in closest:
      elem_сh = list(elem)
      elems.append(elem_сh)
    for elem in elems:
      for el in elem:
        if elem.index(el) == 0:
          elem.append(P(el))
    result = sorted(elems, key=itemgetter(2), reverse=True)
    closest = result[0]
    return closest

In [ ]:
get_closest_hybrid_match('крйг', X, vec)

['крейг', 0.8, 3.491566605803488e-06]

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

Создаем словарь правильных слов

In [ ]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [ ]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

Создаем словарь правильных слов

In [ ]:
vocab_l = list(vocab)

Создаем словарь удалений


In [ ]:
def edits(word):
  letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
  splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
  deletes    = [L + R[1:]               for L, R in splits if R]
  return(deletes)

In [ ]:
def edits2(word): 
    return list(e2 for e1 in edits(word) for e2 in edits(e1))

In [ ]:
changes = []
for word in vocab:
  changes.append(edits(word) + edits2(word))

In [ ]:
ch_dict = []
for change, word in zip(changes, vocab_l):
  ch_dict.append(dict.fromkeys(change, word))

Генерируем варианты ошибок и ищем исправление

In [ ]:
def generate_answer(word):
  options = edits(word) + edits2(word)
  candidates = []
  for elem in ch_dict:
    for opt, key in zip(options, elem):
      if opt == key:
        candidates.append(elem[key])
      elif opt == elem[key]:
        candidates.append(elem[key])
      else:
        continue
  almost_answer = []
  for cand in candidates:
    cand = [cand, P(cand)]
    almost_answer.append(cand)
  answer = sorted(almost_answer, key=itemgetter(1), reverse=True)
  return answer[0]

In [ ]:
generate_answer('сонце')

['конце', 0.00037068798798280367]

In [ ]:
generate_answer('чаащ')

['чаще', 8.030603193348022e-05]

## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса

Byte-Pair Encoding - одна из опций для токенизации текста. В таком случае при токенизации опираются не на какое-то правило, а на соственно статистику по данным. Это особенно полезно в случае, если при обработке встречаются несловарные слова. 

Начнем с того, что при создании токенизатора рассматриваются не только слова, но и "подслова", в основном, продуктивные аффиксы. Таким образом, несловарное слово может быть представлено как последовательность подслов. 

Работа алгоритма byte-pair encoding начанается с того, что на основании текстового корпуса выясняется, какие элементы из словаря отдельных символов чаще всего встречаются вместе. После этого все соседствующие условные A и B заменяются на AB. И таким образом последовательности постеменно удлиняются (до k элементов (k устанавливается в алгоритме)). Результирующий словарь состоит из исходного набора символов плюс k новых символов. 

После обучения алгоритм запускается на тестовых данных и работает по подобному принципу, однако уже опираясь на информацию об устойчивых последовательностях из обучающих данных. Если же попадается неизвестное слово, то, скорее всего, оно будет представлено как несколько частей, внутри которых была статистически обнаружена связь. 